#### 产生背景
早先在hadoop2.0时期, NameNode存在单点故障, 主要表现为: 
* 1. 当namenode发生宕机等突发事件时, 整个hdfs集群变得不可用  
* 2. 当namenode所在机器需要软件或硬件升级时, 需要一定时间的集群下线(cluster downtime)

#### HA架构
* 典型的HA架构中, 急群众存在2个namenode, 任一时刻, 只有一台namenode处于活跃状态, 另一台处于standby状态(备用)
* 为了保持 active node 和 standby node 的状态一致性, 2者都会和一组 journal nodes 沟通;   
  每当 active node 的 namespace 发生改变, 它都会持久性的记录一条日志( edit log )到这组 journal nodes 中的大部分机器;   
  而 standby node 负责读取这些 edit log 的变化, 并将这些变化应用于自身, 从而使得故障发生时, active node 和 standby node 处于一致性状态.
  
#### 硬件资源
* active namenode 和 standby namenode 应该拥有相同的硬件资源
* journal node 的线程是非常轻量级的, 因此可以和其它 hadoop 守护线程放在同一台机器, 像是 jobTracker 或者 YARN ResourceManager

#### 自动失败
自动失败是在 active node 发生故障后, 自动把 standby node 提成 active node 的过程; 自动失败需要增加两个新的组件:  zookeeper 和ZKFailoverController     
zookeeper:   
* **Failure detection - 失败检测** : 集群中每个namenode节点都和 zookeeper 保持一个持久化session (persist session). 一旦active node宕机, 其和zookeeper之间的session将会过期, 并通知 standby node 的 failover应该触发了
* **Active Namenode 选举** : zookeeper提供了一个简单的机制负责重新选择active node. 当当前active node宕机后, 其它某个节点会持有一个排它锁表示自己将成为下一个active node    


ZKFailoverController:  
zkfc是一个zookeeper client, 负责以下几个事情
* **Health monitoring-健康监控** : zkfc进程将定期ping本地的namenode, 如果namenode能回应, 则说明本地namenode处于健康状态
* **zookeeper session管理器** : 维持zookeeper session, 只要本地namenode是健康状态, zkfc就会在zookeeper中持有一个znode节点, 这个节点时临时节点(ephemeral node); 如果session过期, 这个节点将被删除
* **基于zookeeper的选举** : zookeeper会告知该zkfc是否切换为active, 如果是, 则控制本地namenode状态切换到active